# Solutions

In [ ]:
import numpy as np
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
# import netcdf file
file_path = 'path/to/your/folder/TREFHT_EU_10-members_19701999_20702099.nc'
ds1 = xr.open_dataset(file_path).load()

### Main Exercise

In [ ]:
# select days to plot and put them into a list (here, I'm selecting a 6-day period in spring of member 'r6i1231p1f1')

data_list = [ds1.TREFHT.sel(member_id='r6i1231p1f1',time='1985-03-28'),
             ds1.TREFHT.sel(member_id='r6i1231p1f1',time='1985-03-29'),
             ds1.TREFHT.sel(member_id='r6i1231p1f1',time='1985-03-31'),
             ds1.TREFHT.sel(member_id='r6i1231p1f1',time='1985-04-01'),
             ds1.TREFHT.sel(member_id='r6i1231p1f1',time='1985-04-02'),
             ds1.TREFHT.sel(member_id='r6i1231p1f1',time='1985-04-03')]

In [ ]:
# plot
sns.set(style='whitegrid', font_scale=1)
projection=ccrs.PlateCarree()
fig, axes = plt.subplots(nrows=2,ncols=3, figsize=(19,8),  dpi=100, subplot_kw=dict(projection=ccrs.PlateCarree())) # step 1 completed

my_levels = [-14,-10,-6,-2,2,6,10,14]
# or, a cleaner alternative
my_levels = np.arange(-14, 15, 4)

for ax, data in zip(axes.flat, data_list):
    data_C = data - 273.15 # step 2 completed
    p = data_C.plot(ax=ax, cmap='RdBu_r', levels=my_levels, add_colorbar=False) # try with data_C.plot.contourf() instead
    ax.set_extent([3, 33, 54, 72], crs=ccrs.PlateCarree()) # step 3 completed
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.6)    # step 4 completed
    gl.top_labels= False
    gl.right_labels = False
    ax.coastlines(linestyles='-')
    ax.add_feature(cfeature.BORDERS, linewidth=.5) # step 5 completed
    ax.set_aspect(1.2) # optional - stretch maps in the vertical direction
    ax.set_title(np.datetime_as_string(data.time.values.astype('datetime64[ns]'), unit='D'), fontsize=13) # some cosmetic change to give it a %Y-%m-%d format - step 6 completed

plt.colorbar(p, ax=axes.flat, shrink=.9, label="temperature (°C)", ticks=my_levels, orientation='vertical', pad=.04); # step 7 completed
plt.suptitle('Absolute temperature over Scandinavia (Data: CESM2-LENS, member r6i1231p1f1)', fontsize=18, x=.45);

### Advanced exercise

In [ ]:
import datetime as dt

hist_DJF = ds1.TREFHT.sel(time=ds1.time.dt.month.isin([11,12,1]) & (ds1.time.dt.year >= 1970) & (ds1.time.dt.year <= 1999)) # select winter months and historical period
fut_DJF  = ds1.TREFHT.sel(time=ds1.time.dt.month.isin([11,12,1]) & (ds1.time.dt.year >= 2070) & (ds1.time.dt.year <= 2099)) # select winter months and future scenario

# select Iberian region and compute the mean in time
hist_DJF_IB = hist_DJF.sel(lon=slice(-10,3.5),lat=slice(36,44)).mean(dim='time')
fut_DJF_IB  = fut_DJF.sel(lon=slice(-10,3.5),lat=slice(36,44)).mean(dim='time')

In [ ]:
hist_DJF_IB.values.flatten().mean()

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(7.5,4.5))

# plot boxplot
box = axes[0].boxplot([hist_DJF_IB.values.flatten(), fut_DJF_IB.values.flatten()],
                       labels=['hist','fut'],
                       widths=0.5, patch_artist=True,
                       showmeans=False, showfliers=True,
                       medianprops={"color": "gray", "linewidth": 2},
                       boxprops={"edgecolor": "gray",
                                 "linewidth": 0.5, "alpha": 0.7},
                       whiskerprops={"color": "gray", "linewidth": 1.5},
                       capprops={"color": "gray", "linewidth": 1.5})
box['boxes'][0].set_facecolor('#95c8c6')
box['boxes'][1].set_facecolor('#93b8a7')
axes[0].set_ylim(278,295);
axes[0].set_ylabel('Temperature (K)')

# plot histogram
axes[1].hist([hist_DJF_IB.values.flatten(), fut_DJF_IB.values.flatten()],
             color=['#95c8c6','#93b8a7'], edgecolor='gray', bins=15, histtype="stepfilled",
             density=True, alpha=.7)
sns.kdeplot(hist_DJF_IB.values.flatten(), color='#72aba9', ax=axes[1], lw=3)
sns.kdeplot(fut_DJF_IB.values.flatten(), color='#93b8a7', ax=axes[1], lw=3)

# calculate and plot mean of distribution
hist_mean = hist_DJF_IB.values.flatten().mean()
fut_mean  = fut_DJF_IB.values.flatten().mean()
axes[1].axvline(hist_mean, c='#40706f', linestyle='--', linewidth=2)
axes[1].axvline(fut_mean, c='#5c806f', linestyle='--', linewidth=2)
plt.text(276, .145, f'historical - future = +{fut_mean - hist_mean:.1f} K', fontsize=11,
        bbox=dict(facecolor='white', edgecolor='lightgray', boxstyle='round'))
axes[1].set_ylim(0,0.16);

# add title
plt.suptitle('Winter near-surface temperatures (K) over the Iberian Peninsula\n' +
             'historical (1970-1999) vs. future (2070-2099)\nData: CESM2-LENS', fontsize=16);

fig.tight_layout()

There's a lot going on in this plot to give it this final look, not all of which was covered in the course.

Some of it could have no doubt been done better, in a more efficient and elegant way.

But it also serves to demonstrate how many small tweaks can be needed to get the visualisation *just* the way you want it!